In [3]:
import pandas as pd
import geopandas as gpd

import requests
import json

In [4]:
headers = {
    "token": "QBzqRxBKUHUOpoLVBhEeXsYmSsRciZwk"
}

In [14]:
url = "https://api.weather.gov/stations?state=CA"

In [15]:
response = requests.get(url,headers=headers)

In [16]:
response.status_code

200

In [17]:
data = response.json()

11/4 Looking back through this, I have no idea what's going on. `data` is created above - where is stations.geojson coming from?

In [9]:
with open("stations.geojson", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [10]:
metadata_gdf = gpd.GeoDataFrame.from_features(data["features"],crs="EPSG:4326")

In [11]:
metadata_gdf

,geometry,@id,@type,elevation,stationIdentifier,name,timeZone,forecast,county,fireWeatherZone
0,POINT (-121.33943 36.78921),https://api.weather.gov/stations/000PG,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 129.2352}",000PG,Southside Road,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ528,https://api.weather.gov/zones/county/CAC069,https://api.weather.gov/zones/fire/CAZ528
1,POINT (-117.8552 34.11814),https://api.weather.gov/stations/000SE,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 242.9256}",000SE,SCE South Hills Park,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548
2,POINT (-117.73333 34.1255),https://api.weather.gov/stations/001SE,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 409.0416}",001SE,SCE Glen Way,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548
3,POINT (-121.69699 36.63359),https://api.weather.gov/stations/002PG,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 25.2984}",002PG,Reservation Road,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ528,https://api.weather.gov/zones/county/CAC053,https://api.weather.gov/zones/fire/CAZ528
4,POINT (-117.90908 33.96833),https://api.weather.gov/stations/002SE,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 196.9008}",002SE,SCE Native Ave,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548
...,...,...,...,...,...,...,...,...,...,...
495,POINT (-119.3879 34.72528),https://api.weather.gov/stations/254SE,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 1034.4912}",254SE,SCE Maricopa Highway,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ377,https://api.weather.gov/zones/county/CAC111,https://api.weather.gov/zones/fire/CAZ377
496,POINT (-121.68195 37.07418),https://api.weather.gov/stations/255PG,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 333.1464}",255PG,Uvas Hill,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ512,https://api.weather.gov/zones/county/CAC085,https://api.weather.gov/zones/fire/CAZ512
497,POINT (-118.82296 34.11402),https://api.weather.gov/stations/255SE,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 324.0024}",255SE,SCE Lobo Canyon Rd,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ371,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ371
498,POINT (-120.25409 37.92109),https://api.weather.gov/stations/256PG,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 721.7664}",256PG,Apple Colony Road,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ067,https://api.weather.gov/zones/county/CAC109,https://api.weather.gov/zones/fire/CAZ220


Elevation is in meters. I need to remove the nested feature in order to be able to reliably convert to geojson

In [12]:
metadata_gdf.iloc[0]['elevation']

{'unitCode': 'wmoUnit:m', 'value': 129.2352}

In [13]:
metadata_gdf['elevation(m)'] = [metadata_gdf.iloc[i]['elevation']['value'] for i in range(len(metadata_gdf))]

In [14]:
metadata_gdf.drop(columns=['elevation'],inplace=True)

In [15]:
metadata_gdf.head()

,geometry,@id,@type,stationIdentifier,name,timeZone,forecast,county,fireWeatherZone,elevation(m)
0,POINT (-121.33943 36.78921),https://api.weather.gov/stations/000PG,wx:ObservationStation,000PG,Southside Road,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ528,https://api.weather.gov/zones/county/CAC069,https://api.weather.gov/zones/fire/CAZ528,129.2352
1,POINT (-117.8552 34.11814),https://api.weather.gov/stations/000SE,wx:ObservationStation,000SE,SCE South Hills Park,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,242.9256
2,POINT (-117.73333 34.1255),https://api.weather.gov/stations/001SE,wx:ObservationStation,001SE,SCE Glen Way,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,409.0416
3,POINT (-121.69699 36.63359),https://api.weather.gov/stations/002PG,wx:ObservationStation,002PG,Reservation Road,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ528,https://api.weather.gov/zones/county/CAC053,https://api.weather.gov/zones/fire/CAZ528,25.2984
4,POINT (-117.90908 33.96833),https://api.weather.gov/stations/002SE,wx:ObservationStation,002SE,SCE Native Ave,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,196.9008


In [16]:
metadata_gdf.to_file('../../Data/Outputs/meta.geojson', driver='GeoJSON')

### Clipping

I used QGIS to quickly clip to the extent of the LA city boundaries layer. And now, I can import the clipped version

In [19]:
noaa_meta = gpd.read_file("../../Data/Outputs/LA_NOAA_Metadata.geojson")

In [20]:
noaa_meta.head()

,fid,@id,@type,stationIdentifier,name,timeZone,forecast,county,fireWeatherZone,elevation(m),geometry
0,1,https://api.weather.gov/stations/000SE,wx:ObservationStation,000SE,SCE South Hills Park,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,242.9256,POINT (-117.8552 34.11814)
1,2,https://api.weather.gov/stations/001SE,wx:ObservationStation,001SE,SCE Glen Way,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,409.0416,POINT (-117.73333 34.1255)
2,3,https://api.weather.gov/stations/002SE,wx:ObservationStation,002SE,SCE Native Ave,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,196.9008,POINT (-117.90908 33.96833)
3,4,https://api.weather.gov/stations/003SE,wx:ObservationStation,003SE,SCE Fullerton Rd,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,181.3560,POINT (-117.91276 33.9698)
4,5,https://api.weather.gov/stations/004SE,wx:ObservationStation,004SE,SCE N Lyman Ave,America/Los_Angeles,https://api.weather.gov/zones/forecast/CAZ548,https://api.weather.gov/zones/county/CAC037,https://api.weather.gov/zones/fire/CAZ548,213.0552,POINT (-117.85 34.0887)


In [21]:
noaa_meta.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   fid                90 non-null     int32   
 1   @id                90 non-null     object  
 2   @type              90 non-null     object  
 3   stationIdentifier  90 non-null     object  
 4   name               90 non-null     object  
 5   timeZone           90 non-null     object  
 6   forecast           90 non-null     object  
 7   county             90 non-null     object  
 8   fireWeatherZone    90 non-null     object  
 9   elevation(m)       90 non-null     float64 
 10  geometry           90 non-null     geometry
dtypes: float64(1), geometry(1), int32(1), object(8)
memory usage: 7.5+ KB


#### Example: Pulling in readings from station `000SE`

In [5]:
url = "https://api.weather.gov/stations/000SE/observations"
response = requests.get(url,headers=headers)
response.status_code
data = response.json()
with open("stations.geojson", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
gdf2 = gpd.GeoDataFrame.from_features(data["features"],crs="EPSG:4326")

In [6]:
gdf2['timestamp'] = pd.to_datetime(gdf2['timestamp'])

In [7]:
gdf2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   geometry                   982 non-null    geometry           
 1   @id                        982 non-null    object             
 2   @type                      982 non-null    object             
 3   elevation                  982 non-null    object             
 4   station                    982 non-null    object             
 5   stationId                  982 non-null    object             
 6   stationName                982 non-null    object             
 7   timestamp                  982 non-null    datetime64[ns, UTC]
 8   rawMessage                 982 non-null    object             
 9   textDescription            982 non-null    object             
 10  icon                       0 non-null      object             
 11

In [9]:
gdf2.iloc[0]['@id']

'https://api.weather.gov/stations/000SE/observations/2025-11-04T18:40:00+00:00'

In [10]:
gdf2.drop(columns=['icon'],inplace=True)

In [12]:
gdf2.columns

Index(['geometry', '@id', '@type', 'elevation', 'station', 'stationId',
       'stationName', 'timestamp', 'rawMessage', 'textDescription',
       'presentWeather', 'temperature', 'dewpoint', 'windDirection',
       'windSpeed', 'windGust', 'barometricPressure', 'seaLevelPressure',
       'visibility', 'maxTemperatureLast24Hours', 'minTemperatureLast24Hours',
       'precipitationLast3Hours', 'relativeHumidity', 'windChill', 'heatIndex',
       'cloudLayers'],
      dtype='object')

In [13]:
gdf2.iloc[0]['@type']

'wx:ObservationStation'

In [8]:
gdf2.head()

,geometry,@id,@type,elevation,station,stationId,stationName,timestamp,rawMessage,textDescription,...,barometricPressure,seaLevelPressure,visibility,maxTemperatureLast24Hours,minTemperatureLast24Hours,precipitationLast3Hours,relativeHumidity,windChill,heatIndex,cloudLayers
0,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 242.9}",https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-11-04 18:40:00+00:00,,,...,"{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual...","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:mm', 'value': None, 'qua...","{'unitCode': 'wmoUnit:percent', 'value': 69.63...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...",[]
1,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 242.9}",https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-11-04 18:30:00+00:00,,,...,"{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual...","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:mm', 'value': None, 'qua...","{'unitCode': 'wmoUnit:percent', 'value': 70.75...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...",[]
2,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 242.9}",https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-11-04 18:20:00+00:00,,,...,"{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual...","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:mm', 'value': None, 'qua...","{'unitCode': 'wmoUnit:percent', 'value': 72.42...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...",[]
3,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 242.9}",https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-11-04 18:10:00+00:00,,,...,"{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual...","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:mm', 'value': None, 'qua...","{'unitCode': 'wmoUnit:percent', 'value': 72.20...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...",[]
4,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 242.9}",https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-11-04 17:30:00+00:00,,,...,"{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual...","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:mm', 'value': None, 'qua...","{'unitCode': 'wmoUnit:percent', 'value': 79.15...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...",[]


In [93]:
gdf2.columns

Index(['geometry', '@id', '@type', 'elevation', 'station', 'stationId',
       'stationName', 'timestamp', 'rawMessage', 'textDescription',
       'presentWeather', 'temperature', 'dewpoint', 'windDirection',
       'windSpeed', 'windGust', 'barometricPressure', 'seaLevelPressure',
       'visibility', 'maxTemperatureLast24Hours', 'minTemperatureLast24Hours',
       'precipitationLast3Hours', 'relativeHumidity', 'windChill', 'heatIndex',
       'cloudLayers'],
      dtype='object')

In [94]:
gdf2[['elevation','temperature','dewpoint','windDirection','windSpeed','windSpeed','windGust','barometricPressure','seaLevelPressure','visibility']]

,elevation,temperature,dewpoint,windDirection,windSpeed,windSpeed,windGust,barometricPressure,seaLevelPressure,visibility
0,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 25.84, '...","{'unitCode': 'wmoUnit:degC', 'value': 4.95, 'q...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.772,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.772,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 5.76, ...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual..."
1,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 27.17, '...","{'unitCode': 'wmoUnit:degC', 'value': 4.82, 'q...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km_h-1', 'value': 1.224,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 1.224,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 5.184,...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual..."
2,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 27.33, '...","{'unitCode': 'wmoUnit:degC', 'value': 4.94, 'q...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.088,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.088,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 7.524,...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual..."
3,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 27.72, '...","{'unitCode': 'wmoUnit:degC', 'value': 5.7, 'qu...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.016,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.016,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 6.696,...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual..."
4,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 28.39, '...","{'unitCode': 'wmoUnit:degC', 'value': 6.51, 'q...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.484,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 2.484,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 5.652,...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual..."
...,...,...,...,...,...,...,...,...,...,...
1007,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 19.33, '...","{'unitCode': 'wmoUnit:degC', 'value': 11.76, '...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km_h-1', 'value': 1.512,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 1.512,...","{'unitCode': 'wmoUnit:km_h-1', 'value': None, ...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual..."
1008,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 19.67, '...","{'unitCode': 'wmoUnit:degC', 'value': 11.78, '...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km_h-1', 'value': 1.368,...","{'unitCode': 'wmoUnit:km_h-1', 'value': 1.368,...","{'unitCode': 'wmoUnit:km_h-1', 'value': None, ...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual..."
1009,"{'unitCode': 'wmoUnit:m', 'value': 242.9}","{'unitCode': 'wmoUnit:degC', 'value': 19.94, '...","{'unitCode': 'wmoUnit:degC', 'value': 11.98, '...","{'unitCode': 'wmoUnit:degree_(angle)', 'value'...","{'unitCode': 'wmoUnit:km

In [95]:
type(gdf2[gdf2.columns[3]][0])

dict

Finding the names of columns with nested features

In [96]:
nested_columns = []

for i in range(len(gdf2.columns)):
    if type(gdf2[gdf2.columns[i]][0]) == dict:
        nested_columns.append(gdf2.columns[i])

In [97]:
nested_columns

['elevation',
 'temperature',
 'dewpoint',
 'windDirection',
 'windSpeed',
 'windGust',
 'barometricPressure',
 'seaLevelPressure',
 'visibility',
 'maxTemperatureLast24Hours',
 'minTemperatureLast24Hours',
 'precipitationLast3Hours',
 'relativeHumidity',
 'windChill',
 'heatIndex']

In [98]:
[gdf2.iloc[0][f"{column}"]['unitCode'] for column in nested_columns]

['wmoUnit:m',
 'wmoUnit:degC',
 'wmoUnit:degC',
 'wmoUnit:degree_(angle)',
 'wmoUnit:km_h-1',
 'wmoUnit:km_h-1',
 'wmoUnit:Pa',
 'wmoUnit:Pa',
 'wmoUnit:m',
 'wmoUnit:degC',
 'wmoUnit:degC',
 'wmoUnit:mm',
 'wmoUnit:percent',
 'wmoUnit:degC',
 'wmoUnit:degC']

In [99]:
[gdf2.iloc[0][f"{column}"]['unitCode'].split(":")[-1] for column in nested_columns]

['m',
 'degC',
 'degC',
 'degree_(angle)',
 'km_h-1',
 'km_h-1',
 'Pa',
 'Pa',
 'm',
 'degC',
 'degC',
 'mm',
 'percent',
 'degC',
 'degC']

In [100]:
column = nested_columns[0]

This will pull out the values from nested features and create a new column with a name: `column(unit)`, then drop the old nested feature. The new column names are not very Pythonic, but should be ok for now.

In [101]:
for column in nested_columns:
    gdf2[f"{column}({gdf2.iloc[0][f"{column}"]['unitCode'].split(":")[-1]})"] = [gdf2.iloc[i][f"{column}"]['value'] for i in range(len(gdf2))]
    gdf2.drop(columns=[f"{column}"],inplace=True)

In [102]:
gdf2.head()

,geometry,@id,@type,station,stationId,stationName,timestamp,rawMessage,textDescription,presentWeather,...,windGust(km_h-1),barometricPressure(Pa),seaLevelPressure(Pa),visibility(m),maxTemperatureLast24Hours(degC),minTemperatureLast24Hours(degC),precipitationLast3Hours(mm),relativeHumidity(percent),windChill(degC),heatIndex(degC)
0,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 02:30:00+00:00,,,[],...,5.760,None,None,None,None,None,None,26.132239,None,25.161897
1,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 02:20:00+00:00,,,[],...,5.184,None,None,None,None,None,None,23.943461,None,26.567746
2,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 02:10:00+00:00,,,[],...,7.524,None,None,None,None,None,None,23.919091,None,26.410195
3,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 02:00:00+00:00,,,[],...,6.696,None,None,None,None,None,None,24.646499,None,26.689018
4,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 01:50:00+00:00,,,[],...,5.652,None,None,None,None,None,None,25.065196,None,27.171251


In [103]:
gdf2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   geometry                         1012 non-null   geometry           
 1   @id                              1012 non-null   object             
 2   @type                            1012 non-null   object             
 3   station                          1012 non-null   object             
 4   stationId                        1012 non-null   object             
 5   stationName                      1012 non-null   object             
 6   timestamp                        1012 non-null   datetime64[ns, UTC]
 7   rawMessage                       1012 non-null   object             
 8   textDescription                  1012 non-null   object             
 9   presentWeather                   1012 non-null   object           

In [106]:
gdf2.sort_values(by=['timestamp'],ascending=True)

,geometry,@id,@type,station,stationId,stationName,timestamp,rawMessage,textDescription,presentWeather,...,windGust(km_h-1),barometricPressure(Pa),seaLevelPressure(Pa),visibility(m),maxTemperatureLast24Hours(degC),minTemperatureLast24Hours(degC),precipitationLast3Hours(mm),relativeHumidity(percent),windChill(degC),heatIndex(degC)
1011,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-22 02:00:00+00:00,,,[],...,NaN,None,None,None,None,None,None,61.438937,None,NaN
1010,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-22 02:10:00+00:00,,,[],...,NaN,None,None,None,None,None,None,60.429403,None,NaN
1009,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-22 02:20:00+00:00,,,[],...,NaN,None,None,None,None,None,None,60.144486,None,NaN
1008,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-22 02:30:00+00:00,,,[],...,NaN,None,None,None,None,None,None,60.358810,None,NaN
1007,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-22 02:40:00+00:00,,,[],...,NaN,None,None,None,None,None,None,61.566459,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 01:50:00+00:00,,,[],...,5.652,None,None,None,None,None,None,25.065196,None,27.171251
3,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 02:00:00+00:00,,,[],...,6.696,None,None,None,None,None,None,24.646499,None,26.689018
2,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 02:10:00+00:00,,,[],...,7.524,None,None,None,None,None,None,23.919091,None,26.410195
1,POINT (-117.86 34.12),https://api.weather.gov/stations/000SE/observa...,wx:ObservationStation,https://api.weather.gov/stations/000SE,000SE,SCE South Hills Park,2025-10-29 02:20:00+00:00,,,[],...,5.184,None,None,None,None,None,None,23.943461,None,26.567746


In [104]:
1/0

ZeroDivisionError: division by zero

In [ ]:
gdf2['elevation(m)'] = [gdf2.iloc[i]['elevation']['value'] for i in range(len(gdf2))]

In [19]:
url = "https://api.weather.gov/stations/004SE/observations"
response = requests.get(url,headers=headers)
response.status_code
data = response.json()
with open("stations.geojson", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
gdf2 = gpd.GeoDataFrame.from_features(data["features"],crs="EPSG:4326")

In [20]:
gdf2.columns

Index(['geometry', '@id', '@type', 'elevation', 'station', 'stationId',
       'stationName', 'timestamp', 'rawMessage', 'textDescription', 'icon',
       'presentWeather', 'temperature', 'dewpoint', 'windDirection',
       'windSpeed', 'windGust', 'barometricPressure', 'seaLevelPressure',
       'visibility', 'maxTemperatureLast24Hours', 'minTemperatureLast24Hours',
       'precipitationLast3Hours', 'relativeHumidity', 'windChill', 'heatIndex',
       'cloudLayers'],
      dtype='object')

In [21]:
gdf2.iloc[:1]

,geometry,@id,@type,elevation,station,stationId,stationName,timestamp,rawMessage,textDescription,...,barometricPressure,seaLevelPressure,visibility,maxTemperatureLast24Hours,minTemperatureLast24Hours,precipitationLast3Hours,relativeHumidity,windChill,heatIndex,cloudLayers
0,POINT (-117.85 34.09),https://api.weather.gov/stations/004SE/observa...,wx:ObservationStation,"{'unitCode': 'wmoUnit:m', 'value': 213.1}",https://api.weather.gov/stations/004SE,004SE,SCE N Lyman Ave,2025-11-04T22:10:00+00:00,,,...,"{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:Pa', 'value': None, 'qua...","{'unitCode': 'wmoUnit:m', 'value': None, 'qual...","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:degC', 'value': None}","{'unitCode': 'wmoUnit:mm', 'value': None, 'qua...","{'unitCode': 'wmoUnit:percent', 'value': 56.34...","{'unitCode': 'wmoUnit:degC', 'value': None, 'q...","{'unitCode': 'wmoUnit:degC', 'value': 23.60786...",[]


In [ ]:
gdf['elevation(m)'] = [gdf.iloc[i]['elevation']['value'] for i in range(len(gdf))]

## Pulling in observations

Attempting to pull in observations from October 1st, 2025

In [19]:
# Pulling in a generic "latest" reading so we can get the correct format for a time-based URL
url = "https://api.weather.gov/stations/000SE/observations/latest"
response = requests.get(url,headers=headers)
print(response.status_code)
data = response.json()

200


In [17]:
data.keys()

dict_keys(['@context', 'id', 'type', 'geometry', 'properties'])

In [18]:
data['id']

'https://api.weather.gov/stations/000SE/observations/2025-10-29T15:40:00+00:00'

This URL can be used as a template to query based on time/date

In [9]:
url = "https://api.weather.gov/stations/000SE/observations/2025-10-29T15:40:00+00:00"
response = requests.get(url,headers=headers)
print(response.status_code)
data = response.json()

200


In [21]:
data.keys()

dict_keys(['@context', 'id', 'type', 'geometry', 'properties'])

In [22]:
data

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/stations/000SE/observations/2025-10-29T15:40:00+00:00',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-117.86, 34.12]},
 'properties': {'@id':

### Reading in LA AQI Data

We already have air quality data from OpenAQ - now we want further environmental data from the NOAA API. We can try to use the datetime feature from the OpenAQ dataset to pull in data from NOAA on the same time interval

In [15]:
la_aq = gpd.read_file("../../Data/Outputs/la_gdf.gpkg", layer='la_gdf')

In [5]:
la_aq.head()

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,partition_locationid,year,month,day,geometry
0,4715689,14152840,Gramercy Swamp-5805754,2025-10-24T00:00:00-07:00,34.074791,-118.312210,pm25,µg/m³,9.364222,4715689,2025,10,1,POINT (-118.31221 34.07479)
1,3400957,12178340,"Adams Square, Glendale, CA, US-3371044",2025-10-24T00:00:00-07:00,34.135300,-118.244760,pm25,µg/m³,21.987625,3400957,2025,10,1,POINT (-118.24476 34.1353)
2,3632602,12417715,Selma-3602540,2025-10-24T00:00:00-07:00,34.099860,-118.321160,pm25,µg/m³,15.113403,3632602,2025,10,1,POINT (-118.32116 34.09986)
3,3301739,11546715,North Hollywood-3271686,2025-10-24T00:00:00-07:00,34.169239,-118.379672,pm25,µg/m³,17.187875,3301739,2025,10,1,POINT (-118.37967 34.16924)
4,4869634,13437913,Los Angeles - Mid City-4839568,2025-10-24T00:00:00-07:00,34.047822,-118.346382,pm25,µg/m³,11.184014,4869634,2025,10,1,POINT (-118.34638 34.04782)


In [6]:
with open("stations.geojson", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
gdf2 = gpd.GeoDataFrame.from_features(data["features"],crs="EPSG:4326")

NameError: name 'data' is not defined

In [13]:
gdf_temp = gpd.GeoDataFrame(crs="EPSG:4326")

ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'

In [12]:
gdf_test = pd.DataFrame()

In [ ]:
gdf_test.

In [27]:
la_aq['datetime'][20]

'2025-10-24T00:00:00-07:00'

In [25]:
timestamp = "2025-10-29T15:40:00+00:00"

url = f"https://api.weather.gov/stations/000SE/observations/{timestamp}"
response = requests.get(url,headers=headers)
print(f"{response.status_code} for {timestamp}")
data = response.json()

200 for 2025-10-29T15:40:00+00:00


In [31]:
la_aq['datetime'][:20]

0     2025-10-24T00:00:00-07:00
1     2025-10-24T00:00:00-07:00
2     2025-10-24T00:00:00-07:00
3     2025-10-24T00:00:00-07:00
4     2025-10-24T00:00:00-07:00
5     2025-10-24T00:00:00-07:00
6     2025-10-24T00:00:00-07:00
7     2025-10-24T00:00:00-07:00
8     2025-10-24T00:00:00-07:00
9     2025-10-24T00:00:00-07:00
10    2025-10-24T00:00:00-07:00
11    2025-10-24T00:00:00-07:00
12    2025-10-24T00:00:00-07:00
13    2025-10-24T00:00:00-07:00
14    2025-10-24T00:00:00-07:00
15    2025-10-24T00:00:00-07:00
16    2025-10-24T00:00:00-07:00
17    2025-10-24T00:00:00-07:00
18    2025-10-24T00:00:00-07:00
19    2025-10-24T00:00:00-07:00
Name: datetime, dtype: object

In [44]:
la_aq['datetime'][0]

'2025-10-24T00:00:00-07:00'

In [46]:
for timestamp in la_aq['datetime']:
    url = f"https://api.weather.gov/stations/000SE/observations/{timestamp}"
    response = requests.get(url,headers=headers)
    data = response.json()

In [57]:
timestamp = la_aq['datetime'][0]
timestamp2 = la_aq['datetime'][100]
url = f"https://api.weather.gov/stations/000SE/observations?start={timestamp}&end={timestamp2}"
response = requests.get(url,headers=headers)
data = response.json()
data

{'@context': {'@version': '1.1'}, 'type': 'FeatureCollection', 'features': []}

In [54]:
gdf_temp

NameError: name 'gdf_temp' is not defined

In [42]:
https://api.weather.gov/stations/{stationId}/observations?start={2025-10-24T00:00:00-07:00}&end={end}

{'correlationId': '1bdba7b0',
 'title': 'Not Found',
 'type': 'https://api.weather.gov/problems/NotFound',
 'status': 404,
 'detail': 'Not Found',
 'instance': 'https://api.weather.gov/requests/1bdba7b0'}

In [32]:
for timestamp in la_aq['datetime'][:50]:
    url = f"https://api.weather.gov/stations/000SE/observations/{timestamp}"
    response = requests.get(url,headers=headers)
    data = response.json()
    with open("stations.geojson", "w", encoding="utf-8") as f:
        try:
            json.dump(data, f, ensure_ascii=False, indent=2)
            gdf_temp = gpd.GeoDataFrame.from_features(data["features"],crs="EPSG:4326")
            gdf_test = pd.concat([gdf_test, gdf_temp])
            print(f"{response.status_code} for {timestamp}")
        except: 
            print(f"{response.status_code} for {timestamp}")

404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-10-24T00:00:00-07:00
404 for 2025-1

In [4]:
meme = pd.DataFrame()

In [6]:
tset = pd.DataFrame()

In [11]:
memset = pd.concat([tset, meme])

In [1]:
gdf2.head()

NameError: name 'gdf2' is not defined